In [102]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from myDatasets import drivingStyleDataset
from torch.utils.data import DataLoader
import torch.optim as optim
from torch import Tensor
DATAFILE = 'data_raw/drivingStyle60.h5'

In [103]:
#加载训练集和测试集
train = drivingStyleDataset(DATAFILE, train=True, trainNum=50)
test = drivingStyleDataset(DATAFILE, train=False, trainNum=50)
trainDL = DataLoader(train, batch_size=10, shuffle=True, num_workers=0)
testDL = DataLoader(test, batch_size=10)

In [104]:
class CNNNet(nn.Module):
    def __init__(self):
        super(CNNNet, self).__init__()
        #kernel
        self.conv1 = nn.Conv2d(3, 18, 5, 4)
        self.conv2 = nn.Conv2d(18, 27, 3, 2, padding=(1,1))
        self.conv3 = nn.Conv2d(27, 36, 3)
        self.fc1 = nn.Linear(36 * 4 * 4, 270)
        self.fc2 = nn.Linear(270, 120)
        self.fc3 = nn.Linear(120, 40)
        self.fc4 = nn.Linear(40, 3)
    
    def forward(self, x):
        x = F.avg_pool2d(F.relu(self.conv1(x)), 2, padding=(1,1))
        x = F.avg_pool2d(F.relu(self.conv2(x)), 2, padding=(1,1))
        x = F.avg_pool2d(F.relu(self.conv3(x)), 2, padding=(1,1))
        x = x.view(-1, self.num_flat_features(x))
        print(self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x), dim=1)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = CNNNet()
print(net)

CNNNet(
  (conv1): Conv2d(3, 18, kernel_size=(5, 5), stride=(4, 4))
  (conv2): Conv2d(18, 27, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv3): Conv2d(27, 36, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=270, bias=True)
  (fc2): Linear(in_features=270, out_features=120, bias=True)
  (fc3): Linear(in_features=120, out_features=40, bias=True)
  (fc4): Linear(in_features=40, out_features=3, bias=True)
)


In [107]:
optimizer = optim.Adam(net.parameters())
criterion = F.nll_loss

In [108]:
def train(epoch):
    for batch_idx, (data, traget) in enumerate(trainDL):
        optimizer.zero_grad()
        # print(data.float())
        output = net(data.float())
        loss = criterion(output, traget)
        loss.backward()
        #update
        optimizer.step()
        print('epoch %s' % epoch)

def test():
    test_loss = 0
    correct = 0
    for data, target in testDL:
        output = net(data.float())
        #sum up batch loss
        test_loss += criterion(output, target).data[0]
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target

In [109]:
train(1)

576
epoch 1
576
epoch 1
576
epoch 1
576
epoch 1
576
epoch 1
